### 06.03. Import agents into application

In [1]:
#Reuse the agents created in the previous chapters by importing their notebooks
#Ignore / hide the code output and use only the agents

#import the Healthcare assistant agent
%run "code_02_XX A Healthcare assistant agent with ReAct.ipynb" 
print("===============================================================")
#import the appointment scheduling agent
%run "code_05_XX ReAct agent with Workflows - Doctor scheduling.ipynb"

--------
 search_data 
        Use this tool to load data from the following function. It must then be read from
        the corresponding read_search_data function.

        search_data(query: str, lang: str = 'en') -> str

        Search Wikipedia for a page related to the given query.
        Use this tool when `load_data` returns no results.

        Args:
            query (str): the string to search for
        
    
--------
 read_search_data 
        Once data has been loaded from search_data it can then be read using a natural
        language query from this function.

        You are required to pass the natural language query argument when calling this endpoint

        Args:
            query (str): The natural language query used to retreieve information from the index
    
> Running step 48648f15-c477-4982-969d-83786564d942. Step input: Which medication should i take for arthritis?
Thought: The user is asking about medication for arthritis. I can use the query_engine_too

### 06.04 Setup the multi-agent application

In [2]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.tools import FunctionTool
import asyncio

#Sync agent - use as query agent directly
healthcare_agent_tool = QueryEngineTool(
    query_engine= healthcare_agent,
    metadata=ToolMetadata(
        name="Healthcare_Agent",
        description="""
            An agent that can answer questions about diseases,
            symptoms for the diseases and the medications to cure them
            """
    )
)

#Async agent - use an async function to call
async def async_doctors_and_appointments(query: str) -> str:
    response = await scheduling_agent.run(input=query)
    return response.get("response")

#Use a sync wrapper over the async function
def doctors_and_appointments(query: str) -> str :
    """
    An agent that find doctors for specific diseases & specialities
    and then schedule appointments for these doctors.

    Inputs:
    query: This is the original query from the user
    """
    response=asyncio.run(async_doctors_and_appointments(query))
    return response

appointment_scheduling_tool = FunctionTool.from_defaults(doctors_and_appointments)
    
#Create a system prompt
context = """
You are the master agent for a hospital self-service application.
You handle user requests about medications and doctors.

Use only the tools provided to handle the requests and
NOT your own memory.
"""
#Create a Multi-agent
multi_agent = ReActAgent.from_tools(
                [healthcare_agent_tool, appointment_scheduling_tool],
                llm=Settings.llm,
                context=context,
                verbose=True #Set verbose for detailed logs
                )


### 06.05. Execute the multi-agent application

In [3]:
response=multi_agent.chat("Which medication should i take for arthritis?")
print("*******\n Response : ",response.response)

> Running step 13dca8be-57aa-419b-991f-3bd40ed76ef2. Step input: Which medication should i take for arthritis?
Thought: The user is asking about medication for arthritis. I should use the Healthcare_Agent tool to help me answer the question.
Action: Healthcare_Agent
Action Input: {'input': 'medication for arthritis'}
> Running step f5ad0478-e04c-4b9e-9d27-1405a5f26953. Step input: medication for arthritis
Thought: The user is asking about medications for arthritis. I can use the query_engine_tool to provide a list of medications for arthritis.
Action: query_engine_tool
Action Input: {'input': 'arthritis'}
Observation: Methotrexate
> Running step bc872b4c-1d48-4b78-97a5-c9f831f2ef78. Step input: None
Thought: The user has mentioned "Methotrexate" in relation to arthritis. I can provide more information about Methotrexate and its usage for arthritis.
Action: search_data
Action Input: {'query': 'Methotrexate and arthritis'}
Observation: Content loaded! You can now search the information u

In [4]:
response=multi_agent.chat("Which doctors are cardiologists?")
print("*******\n Response : ",response.response)

> Running step 40e20e7e-24f6-4bf5-80e6-6868f3106e12. Step input: Which doctors are cardiologists?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: doctors_and_appointments
Action Input: {'query': 'cardiologist'}
Running step new_user_msg
Step new_user_msg produced event PrepEvent
Running step prepare_chat_history
Step prepare_chat_history produced event InputEvent
Running step handle_llm_input
*** LLM Returned :  thought='The user is requesting information about cardiologists, but I need to use a tool to provide a list of available doctors specializing in cardiology.' action='doctor_tool' action_input={'input': 'cardiologist'}
Step handle_llm_input produced event ToolCallEvent
Running step handle_tool_calls
*** Calling tools :  [ToolSelection(tool_id='fake', tool_name='doctor_tool', tool_kwargs={'input': 'cardiologist'})]
Step handle_tool_calls produced event PrepEvent
Running step prepare_chat_history
Step prepare_chat

In [5]:
response=multi_agent.chat(
    "Find a neurologist and request an appointment for Trevor Hunt at the earliest")

> Running step 2af17b4e-e5b7-4cf3-9443-1a5d994af1e1. Step input: Find a neurologist and request an appointment for Trevor Hunt at the earliest
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: doctors_and_appointments
Action Input: {'query': 'Find a neurologist for appointment for Trevor Hunt at the earliest'}
Running step new_user_msg
Step new_user_msg produced event PrepEvent
Running step prepare_chat_history
Step prepare_chat_history produced event InputEvent
Running step handle_llm_input
*** LLM Returned :  thought='The current language of the user is: English. I need to use a tool to help me answer the question.' action='doctor_tool' action_input={'input': 'neurologist'}
Step handle_llm_input produced event ToolCallEvent
Running step handle_tool_calls
*** Calling tools :  [ToolSelection(tool_id='fake', tool_name='doctor_tool', tool_kwargs={'input': 'neurologist'})]
Step handle_tool_calls produced event PrepEvent
Run